In [6]:
import Pkg; 
Pkg.add("Gurobi"); Pkg.add("JuMP"); Pkg.add("CSV")

using JuMP, Gurobi, CSV

 Resolving package versions...
  Updating `C:\Users\Elton Lin\.julia\environments\v1.1\Project.toml`
 [no changes]
  Updating `C:\Users\Elton Lin\.julia\environments\v1.1\Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `C:\Users\Elton Lin\.julia\environments\v1.1\Project.toml`
 [no changes]
  Updating `C:\Users\Elton Lin\.julia\environments\v1.1\Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `C:\Users\Elton Lin\.julia\environments\v1.1\Project.toml`
 [no changes]
  Updating `C:\Users\Elton Lin\.julia\environments\v1.1\Manifest.toml`
 [no changes]


In [7]:
data= CSV.read("supermarket_data.csv")
distancedata= CSV.read("distmatrix_simple.csv")

#println(distancedata[2][1])

amount_of_items = 60
total_end_bins = amount_of_items + 1
distance_limit = 900
item_limit = 15

item_name = data[1][1:amount_of_items]
price = data[2][1:amount_of_items]
x_pos = data[4][1:amount_of_items]
y_pos = data[5][1:amount_of_items]
push!(price,0)
push!(item_name, "ending point")
push!(x_pos, 0)
push!(y_pos, 0)

distance = Dict()
for j in 1:amount_of_items
    distance_in = Dict()
    distancevals = distancedata[j+1]
    for i in 1:amount_of_items
        distance_in[i] = distancevals[i]
    end
    distance[j] = distance_in 
end

distance_in = Dict()
for i in 1:amount_of_items
    distance_in[i] = distancedata[2][i]
end
distance[total_end_bins] = distance_in




eps = 0.02
# remove pre-solve 

project = Model(with_optimizer(Gurobi.Optimizer, Presolve=0, MIPGap=eps))


#Define Variables
@variable(project, w[1:total_end_bins], Bin)
@variable(project, y[1:total_end_bins] >= 0)
@variable(project, x[1:total_end_bins,1:amount_of_items], Bin)
#x is stored as a array of 10201 where 1:101 represents the x ij for i = 1 and all j. 101:202 for i = 2 and all j
@variable(project, t[1:total_end_bins,1:amount_of_items] >= 0)

#Define Constraints
@constraint(project, y[1] == 0)
@constraint(project, sum(w[i] for i in 1:amount_of_items) <= item_limit)
#valued at 162.989
for i in 1:total_end_bins
    @constraint(project, sum(x[i,j] for j in 1:amount_of_items) == w[i])
end
#binds w[i] to the x[i] values

@constraint(project, sum(x[i,1] for i in 1:total_end_bins) == 1)
#has to start at the start point

@constraint(project, sum(x[total_end_bins,j] for j in 1:amount_of_items) == 1)
#has to end at the end point

for i in 2:total_end_bins
    @constraint(project, sum(x[i,j] for j in 1:amount_of_items) <= 1)
end
for j in 1:amount_of_items
    @constraint(project, sum(x[i,j] for i in 1:total_end_bins) <= 1)
end
#at most 1 thing in and 1 thing out

for i in 1:amount_of_items
    @constraint(project, x[i,i] == 0)
end
#prevents things from going into itself

@constraint(project, y[total_end_bins] <= distance_limit - sum(w[i] for i in 1:amount_of_items)*20)
#distance constraint

# big M method? 
for i in 1:total_end_bins
    for j in 1:amount_of_items
    @constraint(project, x[i,j] <= t[i,j])
    @constraint(project, t[i,j] <= 1000*x[i,j])
    end
end

for i in 1:total_end_bins
    @constraint(project, sum(t[i,j] for j in 1:amount_of_items) == y[i])
end

for j in 1:amount_of_items
    @constraint(project, y[j] + sum(get(get(distance,i,1000),j,1000)*x[i,j] for i in 1:total_end_bins) == sum(t[i,j] for i in 1:total_end_bins) )
end


#Define Objective
totalvalue = sum(price[i]*w[i] for i in 1:total_end_bins)
@objective(project, Max, totalvalue)

optimize!(project)

println("money earned ", value.(totalvalue))
println("total distance traveled ", value.(y[total_end_bins]))

for i in 1:total_end_bins
    if value.(w[i]) == 1
        println("w is 1 at item : ", i)
    end
end

for i in 1:total_end_bins
    for j in 1:amount_of_items
        if value.(x[i,j]) != 0
            println("path x is taken where " , j, " goes to ", i, " with the value : ", value.(x[i,j]))
        end
    end
end

#for i in 1:total_end_bins
#    for j in 1:amount_of_items
 #       if value.(x[get_ij(i,j)])*get(get(distance,i,1000),j,1000) != 0
  #          println("x*dist is where ", j, " goes to " , i,  " with the value : ", value.(x[get_ij(i,j)])*get(get(distance,i,100),j,100))
   #     end
    #end
#end

for i in 1:total_end_bins
    if value.(y[i]) != 0
        println("y the distance at : ", i, " is ", value.(y[i]))
    end
end





Academic license - for non-commercial use only
Academic license - for non-commercial use only
Optimize a model with 7687 rows, 7442 columns and 36964 nonzeros
Variable types: 3721 continuous, 3721 integer (3721 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [9e-01, 3e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 9e+02]
Variable types: 3720 continuous, 3722 integer (3721 binary)

Root relaxation: objective 1.473500e+02, 1985 iterations, 0.60 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  147.35000    0   15          -  147.35000      -     -    1s
H    0     0                      34.9800000  147.35000   321%     -    1s
H    0     0                     119.6900000  147.35000  23.1%     -    3s
     0     0  147.35000    0   57  119.69000  147.35000  23.1%     -    4s
     0     0  147.35000    0   17  11